## Objetivo

Avaliar a viabilidade de diferentes modelos de classificação de sentimento utilizando embeddings pré-treinados, com foco em desempenho preditivo e custo computacional, visando selecionar a melhor configuração para produção.

As etapas conduzidas serão:

1) Download e preparação dos dados e modelos base.

2) Estratificação inteligente dos dados de treino e teste com base em clustering semântico, para garantir diversidade e eficiência amostral.

3) Geração dos datasets vetorizados para cada modelo de embedding.

4) Treinamento e avaliação de classificadores clássicos sobre os vetores gerados, com registro de resultados via MLflow.

5) Análise dos resultados preditivos, com ênfase na métrica AUC sobre dados de teste.

6) Avaliação do desempenho computacional dos embedders em ambiente CPU, considerando latência, memória e tempo de processamento.

7) Escolha do modelo final com base em performance preditiva e custo-benefício operacional para produção.


**Aviso: Este notebook não é conclusivo quanto à tarefa final de escolha dos modelos, dado que postumamente optamos por realizar as predições em produção sobre reviews de jogos da Steam, e não sobre os dados da Amazon utilizados nos treinos e testes até aqui. Assim, torna-se necessário um estudo específico de generalização, a ser conduzido no notebook subsequente, com vistas a avaliar a aderência dos modelos ao novo domínio e prevenir falhas oriundas de Data Drift.**

## Imports

In [2]:
import pandas as pd
from tqdm.notebook import tqdm
from kaggle.api.kaggle_api_extended import KaggleApi
from sentence_transformers import SentenceTransformer
import torch
import tarfile
import sys
import os
import mlflow
import time
import psutil
import gc
sys.path.append(os.path.abspath(".."))
from src import reduce_text_df_per_class, PyCaretEmbeddingClassificationTrainer, BinaryClassificationEvaluator

## Obtendo os dados

In [2]:
data_path = '../data/raw/'
if not os.path.exists(data_path):
    os.makedirs(data_path)

kaggle_api = KaggleApi()
kaggle_api.authenticate()
kaggle_api.dataset_download_files('kritanjalijain/amazon-reviews', path=data_path, unzip=True)

tgz_path = os.path.join(data_path, 'amazon_review_polarity_csv.tgz')

# Extrai direto no diretório, removendo o prefixo do caminho
if os.path.exists(tgz_path):
    with tarfile.open(tgz_path, 'r:gz') as tar:
        for member in tar.getmembers():
            member.name = os.path.relpath(member.name, start=member.name.split('/')[0])
            tar.extract(member, path=data_path)
    os.remove(tgz_path)

Dataset URL: https://www.kaggle.com/datasets/kritanjalijain/amazon-reviews


In [3]:
with open('../data/raw/readme.txt', 'r') as f:
    data_description = f.read()
print(data_description)

Amazon Review Polaridy Dataset

Version 3, Updated 09/09/2015

ORIGIN

The Amazon reviews dataset consists of reviews from amazon. The data span a period of 18 years, including ~35 million reviews up to March 2013. Reviews include product and user information, ratings, and a plaintext review. For more information, please refer to the following paper: J. McAuley and J. Leskovec. Hidden factors and hidden topics: understanding rating dimensions with review text. RecSys, 2013.

The Amazon reviews polarity dataset is constructed by Xiang Zhang (xiang.zhang@nyu.edu) from the above dataset. It is used as a text classification benchmark in the following paper: Xiang Zhang, Junbo Zhao, Yann LeCun. Character-level Convolutional Networks for Text Classification. Advances in Neural Information Processing Systems 28 (NIPS 2015).


DESCRIPTION

The Amazon reviews polarity dataset is constructed by taking review score 1 and 2 as negative, and 4 and 5 as positive. Samples of score 3 is ignored. In th

## Análise e Limpeza

Verificando tamanho dos datasets, se as classes estão balanceadas, e abrindo o head com colunas renomeadas.

In [3]:
columns = ['class', 'review_title', 'review_text']
df_train = pd.read_csv('../data/raw/train.csv', header=None, names=columns)
df_test = pd.read_csv('../data/raw/test.csv', header=None, names=columns)

print(df_train.shape)
print(df_test.shape)
print(df_train['class'].value_counts())
print(df_test['class'].value_counts())
df_train.head(10)

(3600000, 3)
(400000, 3)
class
2    1800000
1    1800000
Name: count, dtype: int64
class
2    200000
1    200000
Name: count, dtype: int64


,class,review_title,review_text
0,2,Stuning even for the non-gamer,This sound track was beautiful! It paints the ...
1,2,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...
2,2,Amazing!,This soundtrack is my favorite music of all ti...
3,2,Excellent Soundtrack,I truly like this soundtrack and I enjoy video...
4,2,"Remember, Pull Your Jaw Off The Floor After He...","If you've played the game, you know how divine..."
5,2,an absolute masterpiece,I am quite sure any of you actually taking the...
6,1,Buyer beware,"This is a self-published book, and if you want..."
7,2,Glorious story,I loved Whisper of the wicked saints. The stor...
8,2,A FIVE STAR BOOK,I just finished reading Whisper of the Wicked ...
9,2,Whispers of the Wicked Saints,This was a easy to read book that made me want...


In [4]:
#Verificando nulos:
print(df_train.isnull().sum())
print(df_test.isnull().sum())

class             0
review_title    207
review_text       0
dtype: int64
class            0
review_title    24
review_text      0
dtype: int64


Como pretendemos processar texto com arquiteturas conhecidas, seguem alguns procedimentos úteis.

In [5]:
# Concatenando review_title e review_text
df_train['text'] = df_train['review_title'] + ' ' + df_train['review_text']
df_test['text'] = df_test['review_title'] + ' ' + df_test['review_text']
df_train.drop(columns=['review_title', 'review_text'], inplace=True)
df_test.drop(columns=['review_title', 'review_text'], inplace=True)

#Vamos trocar os valores 1 e 2 em class por 0 e 1, respectivamente,visando o retorno comum de classifiers binários.
df_train['class'] = df_train['class'].replace(1, 0)
df_train['class'] = df_train['class'].replace(2, 1)
df_test['class'] = df_test['class'].replace(1, 0)
df_test['class'] = df_test['class'].replace(2, 1)

df_train.head(10)


,class,text
0,1,Stuning even for the non-gamer This sound trac...
1,1,The best soundtrack ever to anything. I'm read...
2,1,Amazing! This soundtrack is my favorite music ...
3,1,Excellent Soundtrack I truly like this soundtr...
4,1,"Remember, Pull Your Jaw Off The Floor After He..."
5,1,an absolute masterpiece I am quite sure any of...
6,0,"Buyer beware This is a self-published book, an..."
7,1,Glorious story I loved Whisper of the wicked s...
8,1,A FIVE STAR BOOK I just finished reading Whisp...
9,1,Whispers of the Wicked Saints This was a easy ...


In [6]:
#Verificando se persistem os nulos em df_train e df_test para cada classe, após concatenarmos review_title e review_text
print("Nulos no df_train por classe:")
print(df_train.groupby('class').apply(lambda x: x.isnull().sum()))
print("\nNulos no df_test por classe:")
print(df_test.groupby('class').apply(lambda x: x.isnull().sum()))

Nulos no df_train por classe:
       class  text
class             
0          0   120
1          0    87

Nulos no df_test por classe:
       class  text
class             
0          0    16
1          0     8


Vamos dropar os valores nulos que não poderão ser processados e rebalancear o dataset de treino, embora não seja aqui essencial.

In [7]:
df_train = df_train.dropna(subset=['text'])
df_test = df_test.dropna(subset=['text'])

#Balanceia amostra aleatoriamente min_count exemplos de cada classe
min_count = df_train['class'].value_counts().min()

df_train = (
    df_train.groupby('class')
            .apply(lambda x: x.sample(min_count, random_state=42))
            .reset_index(drop=True)
)
print(f"Nulls em df_train: {df_train.isnull().sum()}")
print(f"Nulls em df_test: {df_test.isnull().sum()}")
print("df_train balanceado:")
print(df_train['class'].value_counts())


Nulls em df_train: class    0
text     0
dtype: int64
Nulls em df_test: class    0
text     0
dtype: int64
df_train balanceado:
class
0    1799880
1    1799880
Name: count, dtype: int64


Apenas por fins de segurança, dado que alguns modelos não suportam janelas muito amplas, vamos obter a estatistica descritiva de length dos reviews, para nos certificar que não ultrapassarão 512 tokens.

In [8]:
df_train['text_length'] = df_train['text'].apply(lambda x: len(x.split()) if isinstance(x, str) else 0)
df_test['text_length'] = df_test['text'].apply(lambda x: len(x.split()) if isinstance(x, str) else 0)
print(df_train['text_length'].describe())
print(df_test['text_length'].describe())
df_train.drop(columns=['text_length'], inplace=True)
df_test.drop(columns=['text_length'], inplace=True)


count    3.599760e+06
mean     7.848389e+01
std      4.283268e+01
min      2.000000e+00
25%      4.200000e+01
50%      7.000000e+01
75%      1.080000e+02
max      2.570000e+02
Name: text_length, dtype: float64
count    399976.000000
mean         78.426493
std          42.798440
min           6.000000
25%          42.000000
50%          70.000000
75%         108.000000
max         230.000000
Name: text_length, dtype: float64


## Baixando modelos base para o projeto

Vamos baixar modelos de embeddings incluindo, quando não a possuem, uma camada de mean pooling para obtê-los todos como Sentence Transformers, visando otimizar a latência do nosso serviço preditivo. 

In [3]:

embedding_dir = "../models/sbert_models"

os.makedirs(embedding_dir, exist_ok=True)

embedding_models = [
    "roberta-base",
    "intfloat/e5-base",
    "BAAI/bge-base-en-v1.5",
    "sentence-transformers/all-MiniLM-L6-v2",
    "sentence-transformers/all-mpnet-base-v2",
    "sentence-transformers/all-MiniLM-L12-v2",
    "thenlper/gte-small",
]

# Desabilita os logs do Hugging Face
#logging.getLogger("sentence_transformers").setLevel(logging.ERROR)
#logging.getLogger("transformers").setLevel(logging.ERROR)

def save_sentence_transformer(model_name, save_dir):
    print(f"Baixando e salvando o modelo {model_name}...")
    # Carrega o modelo diretamente usando sentence-transformers
    model = SentenceTransformer(model_name)
    model.save(os.path.join(save_dir, model_name.split("/")[-1]))  # Salva no diretório correto

for model_name in tqdm(embedding_models, desc="Baixando modelos de embedding"):
    save_sentence_transformer(model_name, embedding_dir)



Baixando modelos de embedding:   0%|          | 0/7 [00:00<?, ?it/s]

Baixando e salvando o modelo roberta-base...


No sentence-transformers model found with name roberta-base. Creating a new one with mean pooling.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Baixando e salvando o modelo intfloat/e5-base...
Baixando e salvando o modelo BAAI/bge-base-en-v1.5...
Baixando e salvando o modelo sentence-transformers/all-MiniLM-L6-v2...
Baixando e salvando o modelo sentence-transformers/all-mpnet-base-v2...
Baixando e salvando o modelo sentence-transformers/all-MiniLM-L12-v2...
Baixando e salvando o modelo thenlper/gte-small...


## Estratificação das amostras de treino e teste
 
Não pretendemos utilizar todos os dados para os treinos que faremos no notebook a seguir, uma vez considerado o custo computacional e temporal do processo.

Optamos, assim, por um método inteligente de estratificação dos dados de treino e teste. Nossa abordagem consistirá em processar uma vez com um dos modelos mais velozes todas as linhas dos datatesets e na sequência clusterizar esses datasets com MiniBatchKMeans, sendo o número de clusters equivalente ao número de linhas resultante para cada dataset, uma vez que o objetivo é coletar exclusivamente o ponto mais próximo do centróide de cada cluster. Com essa abordagem, reamostraremos nossos dados com uma diversidade semântica maximizada para treino e teste, beneficiando-nos da heterogeneidade extracluster.

Abaixo, embedamos ambos os dados de treino e teste.

In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"
sbert = SentenceTransformer("../models/sbert_models/all-MiniLM-L6-v2", device=device)

df_train_embeddings = sbert.encode(
    df_train['text'].tolist(),
    batch_size=256,
    show_progress_bar=True,
    normalize_embeddings=True
)

df_test_embeddings = sbert.encode(
    df_test['text'].tolist(),
    batch_size=256,
    show_progress_bar=True,
    normalize_embeddings=True
)

Batches:   0%|          | 0/14062 [00:00<?, ?it/s]

Batches:   0%|          | 0/1563 [00:00<?, ?it/s]

E aqui clusterizamos os datasets.

Optamos por obter três datasets de treino com vistas a iterar três vezes em diferentes quantidades de dados a pipeline de treinamento e observar o quanto o modelo se beneficia de amostras maiores.

Separamos uma amostra de teste relativamente grande para certificar-nos da precisão das métricas quando da avaliação.

Cf. '../src/data/resample.py/' para detalhes sobre o processo adotado.

In [15]:
train_amazonreviews_sample0005 = reduce_text_df_per_class(
    df=df_train,
    class_column="class",
    embedding_input=df_train_embeddings,
    target_proportion= 0.005,
)

train_amazonreviews_sample0010 = reduce_text_df_per_class(
    df=df_train,
    class_column="class",
    embedding_input=df_train_embeddings,
    target_proportion= 0.01,
)

train_amazonreviews_sample0020 = reduce_text_df_per_class(
    df=df_train,
    class_column="class",
    embedding_input=df_train_embeddings,
    target_proportion= 0.02,
)

test_amazonreviews_sample0050 = reduce_text_df_per_class(
    df=df_test,
    class_column="class",
    embedding_input=df_test_embeddings,
    target_proportion= 0.05,
)


Processing class: 0
Class size: 1799880 → Clusters: 8999
Fitting MiniBatchKMeans...

Processing class: 1
Class size: 1799880 → Clusters: 8999
Fitting MiniBatchKMeans...

Final reduced dataset size: (17998, 2)
class
1    8999
0    8999
Name: count, dtype: int64

Processing class: 0
Class size: 1799880 → Clusters: 17998
Fitting MiniBatchKMeans...

Processing class: 1
Class size: 1799880 → Clusters: 17998
Fitting MiniBatchKMeans...

Final reduced dataset size: (35996, 2)
class
1    17998
0    17998
Name: count, dtype: int64

Processing class: 0
Class size: 1799880 → Clusters: 35997
Fitting MiniBatchKMeans...

Processing class: 1
Class size: 1799880 → Clusters: 35997
Fitting MiniBatchKMeans...

Final reduced dataset size: (71994, 2)
class
0    35997
1    35997
Name: count, dtype: int64

Processing class: 1
Class size: 199992 → Clusters: 9999
Fitting MiniBatchKMeans...

Processing class: 0
Class size: 199984 → Clusters: 9999
Fitting MiniBatchKMeans...

Final reduced dataset size: (19998, 2

In [16]:
train_amazonreviews_sample0005.to_csv(
    "../data/processed/train_amazonreviews_sample0005.csv",
    index=False,
)
train_amazonreviews_sample0010.to_csv(
    "../data/processed/train_amazonreviews_sample0010.csv",
    index=False,
)
train_amazonreviews_sample0020.to_csv(
    "../data/processed/train_amazonreviews_sample0020.csv",
    index=False,
)
test_amazonreviews_sample0050.to_csv(
    "../data/processed/test_amazonreviews_sample0050.csv",
    index=False,
)

## Gerando datasets vetorizados de treino para cada embedder

Como primeira abordagem para o treinamento dos classificadores de sentimento, vamos gerar e salvar versões vetorizadas dos datasets de treino e teste com diferentes embedders. Esses vetores serão usados posteriormente em modelos clássicos de classificação, com o PyCaret facilitando sua configuração.

In [ ]:
models_dir = "../models/sbert_models/"
data_dir = "../data/processed/"
output_dir = os.path.join(data_dir, "embedded_data")
os.makedirs(output_dir, exist_ok=True)

TEXT_COL = "text"
CLASS_COL = "class"

model_names = [m for m in os.listdir(models_dir) if os.path.isdir(os.path.join(models_dir, m))]
dataset_files = [f for f in os.listdir(data_dir) if f.endswith(".csv")]

for model_name in tqdm(model_names, desc="Modelos"):
    model = SentenceTransformer(os.path.join(models_dir, model_name), device="cuda" if torch.cuda.is_available() else "cpu")

    for dataset_file in tqdm(dataset_files, desc=f"Datasets para {model_name}", leave=False):
        df = pd.read_csv(os.path.join(data_dir, dataset_file))

        texts = df[TEXT_COL].astype(str).tolist()
        embeddings = model.encode(texts, batch_size=256, show_progress_bar=False, normalize_embeddings=True)

        emb_df = pd.DataFrame(embeddings, columns=[f"CLS{i}" for i in range(len(embeddings[0]))])
        emb_df[TEXT_COL] = df[TEXT_COL]
        emb_df[CLASS_COL] = df[CLASS_COL]

        output_name = f"{os.path.splitext(dataset_file)[0]}_{model_name}.csv"
        emb_df.to_csv(os.path.join(output_dir, output_name), index=False)

Modelos:   0%|          | 0/7 [00:00<?, ?it/s]

Datasets para all-MiniLM-L12-v2:   0%|          | 0/4 [00:00<?, ?it/s]

Datasets para all-MiniLM-L6-v2:   0%|          | 0/4 [00:00<?, ?it/s]

Datasets para all-mpnet-base-v2:   0%|          | 0/4 [00:00<?, ?it/s]

Datasets para bge-base-en-v1.5:   0%|          | 0/4 [00:00<?, ?it/s]

Datasets para e5-base:   0%|          | 0/4 [00:00<?, ?it/s]

Datasets para gte-small:   0%|          | 0/4 [00:00<?, ?it/s]

Datasets para roberta-base:   0%|          | 0/4 [00:00<?, ?it/s]

## Treinando e avaliando classificadores com embeddings gerados

Nesta etapa, treinamos modelos clássicos de classificação utilizando os datasets vetorizados por diferentes embedders. Utilizamos o PyCaret para facilitar a configuração e comparação dos classificadores com base em desempenho (AUC), salvando os melhores modelos e suas performances sobre os conjuntos teste para análise.

Todos os resultados são registrados via MLflow, conforme o código das classes nos módulos de 'src'.


In [2]:
embedded_data_dir = "../data/processed/embedded_data"

target_column  = 'class'
drop_columns = ['text']

all_files = [f for f in os.listdir(embedded_data_dir) if f.endswith('.csv')]
train_files = [f for f in all_files if f.startswith('train')]
test_files = [f for f in all_files if f.startswith('test')]

def get_embedder_suffix(filename):
    return filename.split('_', 3)[-1]

test_lookup = {get_embedder_suffix(f): f for f in test_files}


for train_file in tqdm(train_files, desc='Datasets de Embeddings'):
    
    suffix = get_embedder_suffix(train_file)
    test_file = test_lookup.get(suffix)

    embedding_model_name = suffix.replace('.csv', '')
    train_dataset_name = train_file.replace('.csv', '')
    df_train = pd.read_csv(os.path.join(embedded_data_dir, train_file))
    df_test = pd.read_csv(os.path.join(embedded_data_dir, test_file))

    
    trainer = PyCaretEmbeddingClassificationTrainer(
        train_dataset=df_train,
        target_column=target_column,
        drop_columns=drop_columns,  
    )

    trained_models = trainer.train()

    run_ids = trainer.log_to_mlflow(
        add_tags={"train_dataset_name": train_dataset_name,
                "embedding_model_name": embedding_model_name,},
    )

    for run_id in run_ids:
        
        with mlflow.start_run(run_id=run_id):
            model_uri = f"runs:/{run_id}/model"
            model = mlflow.sklearn.load_model(model_uri) 

            evaluator = BinaryClassificationEvaluator(
                model=model,
                test_dataset_name=test_file.replace('.csv', ''),
            )

            metrics = evaluator.evaluate_sklearn_model(
                df_test,
                target_column=target_column,
                drop_columns=drop_columns,
            )

            evaluator.log_to_mlflow(metrics)


Datasets de Embeddings:   0%|          | 0/21 [00:00<?, ?it/s]

2025/05/05 00:14:35 INFO mlflow.tracking.fluent: Experiment with name 'pycaret-embeddings-classification' does not exist. Creating a new experiment.


## Análise dos resultados

Vamos obter alguns dos metadados salvos para cada treinamento com vistas a entender qual modelo está performando melhor nos dados de teste.


In [ ]:
client = mlflow.tracking.MlflowClient()
experiment_id = mlflow.get_experiment_by_name("pycaret-embeddings-classification").experiment_id
runs = client.search_runs(experiment_ids=experiment_id)

mlflow_data = []

for run in runs:
    info = {}
    
    # run_id
    info['run_id'] = run.info.run_id

    # Tags
    tags = run.data.tags
    info['model_id'] = tags.get('model_id')
    train_name = tags.get('train_dataset_name')
    parts = train_name.split('_')
    info['train_dataset'] = '_'.join(parts[2:3])
    info['embedding_model'] = tags.get('embedding_model_name')
    info['model_name'] = tags.get('model_name')

    # Métricas
    for metric_name, value in run.data.metrics.items():
        if metric_name.startswith('test'):
            # Procedimento necessário porque não inputamos acima o test_dataset_prefix
            parts = metric_name.split('_')
            new_metric_name = 'amazonreviews' + '_'.join(parts[-1:])
            info[new_metric_name] = value
        else:
            info[metric_name] = value

    mlflow_data.append(info)

mlflow_data = pd.DataFrame(mlflow_data)
print(mlflow_data.shape)
mlflow_data.head(5)


(105, 19)


,run_id,model_id,train_dataset,embedding_model,model_name,Accuracy,AUC,F1,Kappa,MCC,Prec,Recall,amazonreviewsPredictionTime,amazonreviewsTestAccuracy,amazonreviewsTestAUC,amazonreviewsTestF1,amazonreviewsTestPrecision,amazonreviewsTestRecall,TT _Sec_
0,25e222843212478db3bf82975086f4c0,9adf66be-d878-4057-9a73-ce8a148ef6c1,sample0020,roberta-base,SGDClassifier,0.9270,0.9772,0.9263,0.8540,0.8550,0.9364,0.9172,0.017128,0.921792,0.921792,0.919273,0.949867,0.890589,0.581
1,e8a3e0394e8e4bad899abadb6997cfb8,95fc7eb1-55f8-40f1-86bc-e19ff9c8297d,sample0020,roberta-base,LogisticRegression,0.9279,0.9773,0.9263,0.8559,0.8568,0.9479,0.9057,0.029067,0.924342,0.975623,0.922358,0.947196,0.898790,0.780
2,c826f0671e9a40d3a320d41447201f5c,3542dc08-ca02-4370-86c8-e664e8c5f4e3,sample0020,roberta-base,LGBMClassifier,0.9329,0.9803,0.9322,0.8659,0.8661,0.9425,0.9222,0.037168,0.925343,0.978061,0.924294,0.937461,0.911491,3.427
3,bd4af64405d844408e58df3c30c2f1c3,53bcd21c-c306-4b73-8c07-1afc667cb5a0,sample0020,roberta-base,RidgeClassifier,0.9442,0.9849,0.9431,0.8883,0.8889,0.9609,0.9260,0.020077,0.939294,0.939294,0.938074,0.957314,0.919592,0.565
4,6083627879b142a79965e31c6a8f7093,00f0f3e6-2073-4975-a254-f36f70121ef3,sample0020,roberta-base,LinearDiscriminantAnalysis,0.9516,0.9871,0.9512,0.9033,0.9035,0.9608,0.9417,0.028178,0.946745,0.986415,0.946139,0.957029,0.935494,1.077


Levando em consideração que nossos dados estão balanceados e que consideramos testar valores diferentes de threshold para as predições, a melhor métrica para avaliação de performance no nosso caso é a Área Abaixo da Curva (AUC), e particularmente nos dados de teste ('amazonreviewsTestAUC'). 

A métrica 'AUC', que também avaliaremos em segundo plano, refere-se à AUC obtida pelo Pycaret quando efetuou validação cruzada sobre os dados de treino.

Veremos a seguir como cada modelo performou sobre essas métricas, levando em alta consideração o embedding_model subjacente para decidir qual modelo desempenhará melhor em produção.

Também consideraremos o dataset de treino utilizado por cada modelo sob a ótica do sample size referente (0.5%-2% do dataset original), para entendermos o quanto o aumento das amostras em treino afetou a qualidade das predições em teste.

In [ ]:
columns_to_keep = [
    'run_id',
    'train_dataset',
    'embedding_model',
    'model_name',
    #'amazonreviewsPredictionTime', a diferença do tempo de predição entre os classificadores é muito pequena
    'AUC',
    'amazonreviewsTestAUC'
]

filtered_mlflow_data = mlflow_data[columns_to_keep]
filtered_mlflow_data = filtered_mlflow_data.sort_values(by='amazonreviewsTestAUC', ascending=False)
filtered_mlflow_data.reset_index(inplace=True, drop=True)
filtered_mlflow_data.index = filtered_mlflow_data.index + 1
filtered_mlflow_data.to_excel("../reports/AmazonTrain_Results.xlsx")
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
filtered_mlflow_data


,run_id,train_dataset,embedding_model,model_name,AUC,amazonreviewsTestAUC
1,c604ce41a58f4cfa8402e615984f2e16,sample0020,gte-small,LogisticRegression,0.9948,0.994121
2,9212f59735a3429584ba058798a3f8ad,sample0010,gte-small,LogisticRegression,0.9946,0.994086
3,4e61c3ec196444cb94f92d652a9072a6,sample0020,bge-base-en-v1.5,LogisticRegression,0.9948,0.994035
4,87d75ef72bad4c19946559891b71a013,sample0005,gte-small,LogisticRegression,0.9955,0.993981
5,92262b01288749b6b691712e497c9781,sample0010,bge-base-en-v1.5,LogisticRegression,0.9942,0.993963
6,512a0882ecae4f47b3e524a64f5fc987,sample0020,gte-small,LinearDiscriminantAnalysis,0.9944,0.993794
7,fdbb7d2336d344a582e4d304555711b0,sample0005,bge-base-en-v1.5,LogisticRegression,0.9955,0.993728
8,7494240945774d4885516e5565356223,sample0020,bge-base-en-v1.5,LinearDiscriminantAnalysis,0.9942,0.993675
9,eb3845d7e64d41908941924bf006e325,sample0005,gte-small,LinearDiscriminantAnalysis,0.9950,0.993651
10,766f158e433d4942ad6463409a6f1813,sample0010,gte-small,LinearDiscriminantAnalysis,0.9944,0.993626


Entre os modelos avaliados, destacaram-se as combinações dos embeddings gte-small e bge-base-en-v1.5 com os classificadores Logistic Regression e Linear Discriminant Analysis, que apresentaram AUCs superiores a 0,993 nos dados de teste ('amazonreviewsTestAUC'), com alta estabilidade frente às variações no tamanho da amostra. Isso indica robustez mesmo com conjuntos de treino reduzidos.

Embora tenhamos testado diferentes sample sizes (0,5%, 1% e 2%), os ganhos marginais entre eles foram mínimos entre os melhores modelos, o que justifica manter 0,5% como padrão, priorizando menor custo computacional e preservando a qualidade das predições.

Modelos com embeddings como all-MiniLM e roberta-base mostraram desempenho inferior de forma consistente, independentemente do classificador ou sample size, o que os torna pouco indicados para o contexto atual.

Apesar dos resultados apontarem com clareza os modelos mais promissores em termos de AUC, a escolha final para produção dependerá da análise do custo-benefício computacional dos embeddings, a ser realizada na sequência.

## Análise do Desempenho computacional dos embedders

Considerando que a decisão final sobre os modelos envolve também critérios de custo-benefício computacional, avaliamos agora os principais aspectos de performance dos embedders em ambiente CPU (ambiente que nos estará disponível em produção): tempo de carregamento, latência online, tempo de codificação em lote, uso líquido de RAM, pico de memória e tamanho em disco. A análise será conduzida com diferentes tamanhos de batch e repetida múltiplas vezes para maior robustez nas médias. Os resultados serão utilizados como base para ponderar a viabilidade de produção de cada embedding_model.


In [4]:
df_sample = pd.read_csv("../data/processed/train_amazonreviews_sample0005.csv")
df_sample = df_sample.sample(n=1000, random_state=42)
print(df_sample.shape)
df_sample.head(5)

(1000, 2)


,class,text
11364,0,Don't Buy this book light This is a poorly des...
4119,0,modem was in bad condition and did not work! T...
2008,1,The Greatest Superhero Movie of All Time Chris...
12572,1,Depravity driven home... I've recently watched...
4755,1,EVERY SONG ROCKS They did something different!...


In [ ]:
texts = df_sample["text"].tolist()[:1000]
models_dir = "../models/sbert_models/"
model_names = [name for name in os.listdir(models_dir) if os.path.isdir(os.path.join(models_dir, name))]

batch_sizes = [1, 8, 16, 32, 64]
n_runs = 3

# Memória RAM atual (em MB)
def get_memory_usage_mb():
    gc.collect()
    return psutil.Process(os.getpid()).memory_info().rss / (1024 ** 2)

# Função para medir o uso máximo de memória (simulando "pico")
def encode_and_measure_peak_memory(model, texts, bs):
    gc.collect()
    process = psutil.Process(os.getpid())
    mem_before = process.memory_info().rss / (1024 ** 2)

    peak_memory = mem_before
    start_time = time.time()
    
    for t in range(0, len(texts), bs):
        _ = model.encode(texts[t:t + bs], show_progress_bar=False)
        current = process.memory_info().rss / (1024 ** 2)
        peak_memory = max(peak_memory, current)

    total_time = time.time() - start_time
    mem_after = process.memory_info().rss / (1024 ** 2)
    net_mem = max(mem_after - mem_before, 0)

    return total_time, net_mem, peak_memory - mem_before

results = []

def safe_avg(lst, name):
    if all(x == 0 for x in lst):
        print(f"⚠️ AVISO: {name} de '{model_name}' batch_size={bs} foi 0 em todas as execuções. Ignorando média.")
        return None
    return sum(lst) / len([x for x in lst if x != 0])

for model_name in tqdm(model_names, desc="Embedding Models Analisados"):
    model_path = os.path.join(models_dir, model_name)

    # Medir tempo de carregamento
    load_times = []
    for _ in range(n_runs):
        gc.collect()
        start_time = time.time()
        model = SentenceTransformer(model_path, device="cpu")
        load_times.append(time.time() - start_time)
        del model
        gc.collect()

    if all(x == 0 for x in load_times):
        print(f"⚠️ AVISO: load_time de '{model_name}' foi 0 em todas as execuções. Ignorando média.")
        avg_load_time = None
    else:
        avg_load_time = sum(load_times) / len([x for x in load_times if x != 0])

    # Tamanho do modelo em disco
    model_size_bytes = sum(
        os.path.getsize(os.path.join(root, f))
        for root, _, files in os.walk(model_path)
        for f in files
    )
    model_size_mb = model_size_bytes / (1024 ** 2)

    model = SentenceTransformer(model_path, device="cpu")

    for bs in batch_sizes:
        latencies, batch_times, ram_usages_net, ram_usages_peak = [], [], [], []

        for _ in range(n_runs):
            # Latência online
            gc.collect()
            start_time = time.time()
            for t in texts[:10]:
                _ = model.encode(t, show_progress_bar=False)
            latency = (time.time() - start_time) / 10
            latencies.append(latency)

            # RAM líquida + tempo + pico
            try:
                batch_time, ram_net, ram_peak = encode_and_measure_peak_memory(model, texts, bs)
            except Exception as e:
                print(f"Erro ao medir uso de memória para {model_name}, batch {bs}: {e}")
                batch_time, ram_net, ram_peak = 0, 0, 0

            batch_times.append(batch_time)
            ram_usages_net.append(ram_net)
            ram_usages_peak.append(ram_peak)

        results.append({
            "model": model_name,
            "batch_size": bs,
            "load_time_sec": round(avg_load_time, 2) if avg_load_time else None,
            "latency_online_sec": round(safe_avg(latencies, "latency_online_sec"), 4) if safe_avg(latencies, "latency_online_sec") else None,
            "batch_time_sec": round(safe_avg(batch_times, "batch_time_sec"), 2) if safe_avg(batch_times, "batch_time_sec") else None,
            "ram_usage_net_mb": round(safe_avg(ram_usages_net, "ram_usage_net_mb"), 2) if safe_avg(ram_usages_net, "ram_usage_net_mb") else None,
            "ram_usage_peak_mb": round(safe_avg(ram_usages_peak, "ram_usage_peak_mb"), 2) if safe_avg(ram_usages_peak, "ram_usage_peak_mb") else None,
            "model_size_mb": round(model_size_mb, 2)
        })

embedders_info = pd.DataFrame(results)
embedders_info=embedders_info.sort_values("batch_time_sec")
embedders_info.reset_index(inplace=True, drop=True)
embedders_info.index = embedders_info.index + 1
embedders_info.to_excel("../reports/Sbert_CPU_Benchmark.xlsx")
embedders_info

Embedding Models Analisados:   0%|          | 0/7 [00:00<?, ?it/s]

⚠️ AVISO: ram_usage_net_mb de 'all-mpnet-base-v2' batch_size=8 foi 0 em todas as execuções. Ignorando média.
⚠️ AVISO: ram_usage_net_mb de 'all-mpnet-base-v2' batch_size=64 foi 0 em todas as execuções. Ignorando média.
⚠️ AVISO: ram_usage_peak_mb de 'all-mpnet-base-v2' batch_size=64 foi 0 em todas as execuções. Ignorando média.
⚠️ AVISO: ram_usage_net_mb de 'bge-base-en-v1.5' batch_size=64 foi 0 em todas as execuções. Ignorando média.
⚠️ AVISO: ram_usage_net_mb de 'e5-base' batch_size=64 foi 0 em todas as execuções. Ignorando média.
⚠️ AVISO: ram_usage_net_mb de 'gte-small' batch_size=64 foi 0 em todas as execuções. Ignorando média.
⚠️ AVISO: ram_usage_net_mb de 'roberta-base' batch_size=8 foi 0 em todas as execuções. Ignorando média.
⚠️ AVISO: ram_usage_net_mb de 'roberta-base' batch_size=32 foi 0 em todas as execuções. Ignorando média.
⚠️ AVISO: ram_usage_net_mb de 'roberta-base' batch_size=64 foi 0 em todas as execuções. Ignorando média.


,model,batch_size,load_time_sec,latency_online_sec,batch_time_sec,ram_usage_net_mb,ram_usage_peak_mb,model_size_mb
1,all-MiniLM-L6-v2,64,0.04,0.0155,8.66,21.59,22.37,87.57
2,all-MiniLM-L6-v2,16,0.04,0.0159,12.76,25.39,25.39,87.57
3,all-MiniLM-L6-v2,8,0.04,0.0158,12.78,1.17,1.25,87.57
4,all-MiniLM-L12-v2,64,0.06,0.0277,13.33,10.96,11.18,128.19
5,all-MiniLM-L6-v2,32,0.04,0.0157,13.62,37.20,37.20,87.57
6,all-MiniLM-L6-v2,1,0.04,0.0159,13.96,13.50,9.04,87.57
7,gte-small,64,0.12,0.0308,17.70,NaN,0.01,128.25
8,all-MiniLM-L12-v2,32,0.06,0.0279,20.33,62.34,62.84,128.19
9,all-MiniLM-L12-v2,16,0.06,0.0282,20.69,11.46,11.96,128.19
10,all-MiniLM-L12-v2,8,0.06,0.0278,21.15,18.64,18.64,128.19


Apesar do all-MiniLM-L6-v2 apresentar a menor latência online e em batch entre todos os modelos testados, seu desempenho significativamente inferior em AUC o desqualifica para produção em comparação com o alto custo-benefício do gte-small que, mesmo com latência levemente superior, entrega AUCs robustos com baixo consumo de memória relativamente a outros modelos mais pesados.

## Conclusão
Diante das análises conduzidas, tanto sob a ótica da eficácia preditiva quanto sob critérios de viabilidade computacional, evidencia-se como escolha mais prudente para produção a combinação do embedder gte-small com o classificador Logistic Regression treinado a partir da amostra de 0,5% dos dados originais.

Tal decisão se ancora, primeiramente, nos resultados superiores obtidos em termos de AUC nos dados de teste, consistentes mesmo sob variações no volume amostral — atributo que denota elevada robustez e estabilidade preditiva. Em particular, observou-se que ganhos marginais entre amostras de 0,5% a 2% são desprezíveis, o que legitima a adoção do menor sample size como padrão, maximizando a relação custo-benefício do processo.

Adicionalmente, a avaliação de desempenho computacional corroborou essa escolha ao revelar que, embora não seja o modelo mais veloz em latência, o gte-small mantém um equilíbrio exemplar entre tempo de resposta, footprint de memória e tamanho em disco, superando largamente modelos mais custosos e menos eficazes como o roberta-base e o all-MiniLM.

Assim, optamos, enquanto exclusivamente atentos à tarefa de predição de sentimento sobre reviews da Amazon, pela run de id 87d75ef72bad4c19946559891b71a013.